In [ ]:
from numpy.random import seed
seed(1)
import tensorflow
tensorflow.random.set_seed(2)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import scipy as sp
import sklearn
from sklearn.decomposition import TruncatedSVD
from matplotlib import pyplot as plt
from IPython.display import clear_output
import powerlaw
import tensorflow_addons as tfa
import copy
import weightwatcher as ww
import imageio
from datetime import datetime
import io
import cv2
# Suppress the powerlaw package warnings
# "powerlaw.py:700: RuntimeWarning: divide by zero encountered in true_divide"
# "powerlaw.py:700: RuntimeWarning: invalid value encountered in true_divide"
import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)
import random

In [ ]:
# Model / data parameters
num_classes = 10
inputShape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# shuffle training set and its labels accordingly
trainingIndexes = np.arange(0,y_train.shape[0]).tolist()
random.shuffle(trainingIndexes)
x_train = x_train[trainingIndexes,:,:,:]
y_train = y_train[trainingIndexes,:]

In [ ]:
# LENET
model = keras.Sequential(
    [
        keras.layers.InputLayer(input_shape=inputShape),
        layers.Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu'),
        layers.MaxPool2D(strides=2),
        layers.Conv2D(filters=48, kernel_size=(5,5), padding='valid', activation='relu'),
        layers.MaxPool2D(strides=2),
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dense(84, activation='relu'),
        layers.Dense(10, activation='softmax'),
    ]
)
 
model.summary()

In [ ]:
trainingSize = 2000
testSize = 10000
batch_size = 128
epochs = 15
learningRate = .001

In [ ]:
opt = keras.optimizers.Adam(learning_rate=learningRate)
 
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
 
model.fit(x_train[0:trainingSize], y_train[0:trainingSize], batch_size=batch_size, epochs=epochs, validation_split=0)

In [ ]:
# do a deep copy (keras clone) of the model and work with that
modelToSmooth = keras.models.clone_model(model)
modelToSmooth.build(model.input_shape) 
modelToSmooth.compile(loss=model.loss, optimizer=model.optimizer, metrics=["accuracy"])
modelToSmooth.set_weights(model.get_weights())
    
watcher = ww.WeightWatcher(model=modelToSmooth)
watcher.analyze()

In [ ]:
print("Training set accuracy " + str(modelToSmooth.evaluate(x_train[0:trainingSize], y_train[0:trainingSize], verbose=0)[1]))

In [ ]:
print("Test set accuracy " + str(modelToSmooth.evaluate(x_test[0:testSize], y_test[0:testSize], verbose=0)[1]))

In [ ]:
(_, nComponents) = watcher.unifiedSVDSmoothing(methodSelectComponents = "mp_spikes") 
# Other options are: 
# methodSelectComponents = "powerlaw_xmin" ==> default
# methodSelectComponents = "powerlaw_spikes"
# methodSelectComponents = "percentage", percent = 0.49
print("Components selected " + str(nComponents))

In [ ]:
print("Estimated test accuracy " + str(modelToSmooth.evaluate(x_train[0:trainingSize], y_train[0:trainingSize], verbose=0)[1]))